In [ ]:
import pandas as pd
import numpy as np
from datetime import date

In [ ]:
df = pd.read_csv('OKR_Data.csv')

In [ ]:
df.columns

# Data Clean

In [ ]:
df.columns = ['ID', 'NAME','TAG','TEAM','CREATOR','OWNER','PERIOD','START_DATE','END_DATE','DESCRIPTION','ALIGNMENT','METRICS','Target','OBJECTIVE_TYPE','LAST_CHECKIN','PROGRESS','STATUS','LAST_CHECKIN_NOTES','SCORE']
df['PROGRESS'] = df['PROGRESS']/100
df['SCORE'].fillna('0',inplace=True)

In [ ]:
#Clean up hyperlinked id (string)
df['ID'] = (df['ID'].str.split(',"',expand=True)[1].str.replace('")','')).astype(int)
df['ID']

In [ ]:
#Clean up owner from last name, first name, last name, first name...) 
#to (first name last name, first name last name...). 
#Order doesn't matter
df['OWNER'].head(30)

In [ ]:
def name_clean(names):
    
    names = names.split(', ')[::-1]
    
    for n in range(1,len(names[1::2])):
        names[2*n-1] = names[1::2][n-1] + ','
        
    return ' '.join(names)

In [ ]:
df['OWNER'] = df['OWNER'].apply(name_clean)
df['OWNER']

name_arr = []
for x in lines:
    name = ""
    blocks = x.split(",")
    for i, k in zip(blocks[0::2], blocks[1::2]):
        name = name + "{} {},".format(i,k)
    
    name_arr.append(name[:-1])
              
df_owner = pd.DataFrame({
'owner': name_arr
})

print(df_owner)

In [ ]:
##Clean the text format alignment and extract alignment id(s)
align = df['ALIGNMENT'].str.split('Id:',expand = True)
align

In [ ]:
align = align.replace(np.nan,None)

In [ ]:
align.drop(0,axis=1,inplace=True)
align.columns=['a1','a2','a3']

In [ ]:
align[['a1','a2','a3']] = align[['a1','a2','a3']].apply(lambda x: x.str.split(')').str[0])

In [ ]:
align

In [ ]:
#add cleaned alignment id back to df
df['ALIGN_ID'] = align.apply(lambda x: np.nan if x.isnull().all() else ','.join(x.dropna()), axis=1)
df['ALIGN_ID'] = df['ALIGN_ID'].str.split(',')

In [ ]:
#separate as new row
df = df.explode('ALIGN_ID')

In [ ]:
#Convert to int for joins
df['ALIGN_ID'] = df['ALIGN_ID'].fillna(0).astype(int)

# Annual OKR

In [ ]:
## Tech Annual OKR
aokr = df[(df['TEAM'] == 'Company_Name') & (df['PERIOD'].str.contains('Annual'))]
aokr

In [ ]:
df_annual = aokr.merge(aokr,left_on='ALIGN_ID',right_on = 'ID') ##!!annual key results + annual objectives
df_annual

In [ ]:
df_annual.columns #x-->kr, y-->o

In [ ]:
df_annual = df_annual.drop(['OBJECTIVE_TYPE_x','OBJECTIVE_TYPE_y','ALIGN_ID_y','ALIGNMENT_y'],axis=1)

In [ ]:
df_annual.columns = ['AKR_ID', 'AKR_NAME', 'AKR_TAG', 'AKR_TEAM', 'AKR_CREATOR','AKR_OWNER', 'AKR_PERIOD', 'AKR_START_DATE', 'AKR_END_DATE', 'AKR_DESCRIPTION',
       'AKR_ALIGNMENT','AKR_METRICS','AKR_TARGET', 'AKR_LAST_CHECKIN','AKR_PROGRESS', 'AKR_STATUS', 'AKR_LAST_CHECKIN_NOTES','AKR_SCORE',
                        'AKR_ALIGN_REF_ID','AO_ID','AO_NAME','AO_TAG','AO_TEAM','AO_CREATOR','AO_OWNER','AO_PERIOD','AO_START_DATE','AO_END_DATE',
                        'AO_DESCRIPTION','AO_METRICS','AO_TARGET','AO_LAST_CHECKIN', 'AO_PROGRESS', 'AO_STATUS','AO_LAST_CHECKIN_NOTES', 'AO_SCORE']

# Departmental OKRs

In [ ]:
#departmental okr
dokr = df[df['TEAM'] != 'Company_Name']
dokr

In [ ]:
#Departmental quarterly okrs
dokr_q = dokr[dokr['PERIOD'].str.contains('Q')]

In [ ]:
df_dep_q = dokr_q.merge(dokr_q,left_on='ALIGN_ID',right_on = 'ID')#x-->kr,y-->o
df_dep_q

In [ ]:
df_dep_q.columns

In [ ]:
df_dep_q.columns = [ 'DKR_ID', 'DKR_NAME', 'DKR_TAG', 'DKR_TYPE_NAME', 'DKR_CREATOR',
       'DKR_OWNER', 'DKR_PERIOD', 'DKR_START_DATE', 'DKR_END_DATE', 'DKR_DESCRIPTION',
       'DKR_ALIGNMENT','DKR_METRICS', 'DKR_TARGET','DKR_OBJECTIVE_TYPE', 'DKR_LAST_CHECKIN','DKR_PROGRESS', 'DKR_STATUS', 'DKR_LAST_CHECKIN_NOTES','DKR_SCORE',
       'DKR_ALIGN_REF_ID','DO_ID', 'DO_NAME', 'DO_TAG', 'DO_TYPE_NAME', 'DO_CREATOR',
       'DO_OWNER', 'DO_PERIOD', 'DO_START_DATE', 'DO_END_DATE', 'DO_DESCRIPTION',
       'DO_ALIGNMENT', 'DO_METRICS','DO_TARGET','DO_OBJECTIVE_TYPE','DO_LAST_CHECKIN','DO_PROGRESS', 'DO_STATUS','DO_LAST_CHECKIN_NOTES', 'DO_SCORE',
       'DO_ALIGN_REF_ID']

In [ ]:
##annual department objectives
df_dep_a = dokr[dokr['PERIOD'].str.contains('Annual')]
df_dep_a

In [ ]:
df_dep = df_dep_q.merge(df_dep_a,left_on='DO_ALIGN_REF_ID',right_on = 'ID',how ='left')
df_dep.columns

In [ ]:
df_dep = df_dep.rename(columns={'ID':'ADO_ID', 'NAME':'ADO_NAME', 'TAG':'ADO_TAG', 'TEAM':'ADO_TEAM', 'CREATOR':'ADO_CREATOR',
       'OWNER':'ADO_OWNER', 'PERIOD':'ADO_PERIOD', 'START_DATE':'ADO_START_DATE', 'END_DATE':'ADO_END_DATE', 'DESCRIPTION':'ADO_DESCRIPTION', 
       'ALIGNMENT':'ADO_ALIGNMENT','METRICS':'ADO_METRICS', 'Target':'ADO_Target', 'OBJECTIVE_TYPE':'ADO_OBJECTIVE_TYPE', 'LAST_CHECKIN':'ADO_LAST_CHECKIN', 'PROGRESS':'ADO_PROGRESS',
       'STATUS':'ADO_STATUS', 'LAST_CHECKIN_NOTES':'ADO_LAST_CHECKIN_NOTES', 'SCORE':'ADO_SCORE', 'ALIGN_ID':'ADO_ALIGN_REF_ID'})   

In [ ]:
#Join annual okrs with departmental annual objective or departmental objective
#if there's no departmenal annual o, use quarterly o as the key
df_dep['ADO_ID'] = df_dep['ADO_ID'].fillna(0)

In [ ]:
df_dep['Combined_ALIGN_REF_ID'] = np.where(df_dep['ADO_ID'] == 0,df_dep['DO_ALIGN_REF_ID'],df_dep['ADO_ALIGN_REF_ID'])

In [ ]:
df_report = df_annual.merge(df_dep, left_on='AKR_ID', right_on = 'Combined_ALIGN_REF_ID', how ='outer')
df_report['export_date'] = date.today()

In [ ]:
df_report.to_excel('okr_df_report.xlsx',index=False)